In [1]:
# Read in raw ratings data (fromUserId, toUserId, rating)
ratingsCsvRDD = sc.textFile("s3n://insight-spark-after-dark/ratings.csv.gz")

In [2]:
# Convert raw ratings RDD to Json RDD
def rec_tup(rating):
    tokens = rating.split(",")
    return {"fromUserId": tokens[0], "toUserId": tokens[1], "rating": tokens[2]}

ratingsJsonRDD = ratingsCsvRDD.map(rec_tup)
ratingsJsonRDD.take(5)

[{'fromUserId': u'1', 'rating': u'8', 'toUserId': u'133'},
 {'fromUserId': u'1', 'rating': u'6', 'toUserId': u'720'},
 {'fromUserId': u'1', 'rating': u'10', 'toUserId': u'971'},
 {'fromUserId': u'1', 'rating': u'7', 'toUserId': u'1095'},
 {'fromUserId': u'1', 'rating': u'10', 'toUserId': u'1616'}]

In [4]:
# Create SchemaRDD from JsonRDD
ratingsJsonSchemaRDD = sqlCtx.jsonRDD(ratingsJsonRDD)

In [5]:
# Cache the SchemaRDD as we'll be using this heavily moving forward
ratingsJsonSchemaRDD.cache()

DataFrame[_corrupt_record: string]

In [7]:
# Describe the SchemaRDD inferred from the JSON
ratingsJsonSchemaRDD.printSchema()

ratingsJsonSchemaRDD.registerTempTable("ratingsJsonTable")

root
 |-- _corrupt_record: string (nullable = true)



In [8]:
sqlCtx.sql("DESCRIBE ratingsJsonTable")

DataFrame[col_name: string, data_type: string, comment: string]

In [12]:
# Show the top 10 most-active users
mostActiveUsersSchemaRDD = sqlCtx.sql("SELECT fromUserId, count(*) as ct FROM ratingsJsonTable GROUP BY fromUserId")

Py4JJavaError: An error occurred while calling o45.sql.
: org.apache.spark.sql.AnalysisException: cannot resolve 'fromUserId' given input columns _corrupt_record; line 1 pos 65
	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$apply$3$$anonfun$apply$1.applyOrElse(CheckAnalysis.scala:48)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$apply$3$$anonfun$apply$1.applyOrElse(CheckAnalysis.scala:45)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$transformUp$1.apply(TreeNode.scala:250)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$transformUp$1.apply(TreeNode.scala:250)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:50)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:249)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$transformExp